# 分位数VAR模型

## 原理讲解

参考文献：
1. VAR for VaR: Measuring tail dependence using multivariate regression quantiles
2. 分位数向量自回归分布滞后模型及脉冲响应分析

## 使用 statsmodels 库实现

In [1]:
import pandas as pd
import statsmodels.regression.quantile_regression as qr
import statsmodels.api as sm

data = pd.read_excel('../数据/上证指数与沪深300.xlsx')
Y = data['hs300']
X = data['sz']

def lag_list(Y, X, p=1, q=1, exogenous=[]):
    '''
    待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q) + exogenous
    获取自回归分布滞后模型的估计向量

    Parameters
    ----------
    Y : 被估计变量
    X : 估计变量
    p : ADL 模型 Y 的滞后阶数，标量默认为1
    q : ADL 模型 X 的滞后阶数，标量默认为1
    exogenous : 外生变量

    Returns
    -------
    ADLy : ADL 模型被解释变量
    ADLx : ADL 模型解释变量

    '''
    ADLx = pd.DataFrame()
    T = len(Y)
    ADLy = list(Y[max(p, q):T])
    for i in range(1, p+1):
        name = f'y_{i}'
        ADLx[name] = list(Y[max(p, q)-i:T-i])
    for i in range(1, q+1):
        name = f'x_{i}'
        ADLx[name] = list(X[max(p, q)-i:T-i])
    
    # 增加控制变量
    if type(exogenous) == pd.Series:
        ADLx[exogenous.name] = list(exogenous[:0-max(p, q)])
    elif type(exogenous) == pd.DataFrame:
        for name in exogenous.columns:
            ADLx[name] = list(exogenous[name][:0-max(p, q)])
    
    # 增加常数项
    ADLx = sm.add_constant(ADLx)
    return ADLy, ADLx


def qvar(Y, X, P, Q, exogenous=[]):
    '''
    待估计方程：y = c + y(-1) +....+y(-p) + x(-1) + ... + x(-q) + exogenous
               x = c + x(-1) +....+x(-p) + y(-1) + ... + y(-q) + exogenous
    估计 QVAR 模型

    Parameters
    ----------
    Y : 被估计变量
    X : 估计变量
    P : QVAR 模型的滞后阶数
    Q : 分位点
    exogenous : 外生变量

    '''
    ADLy, ADLx = lag_list(Y, X ,P, P, exogenous)
    mod = qr.QuantReg(ADLy, ADLx)
    res = mod.fit(Q)
    beta1 = res.params
    
    ADLy, ADLx = lag_list(X, Y ,P, P, exogenous)
    mod = qr.QuantReg(ADLy, ADLx)
    res = mod.fit(Q)
    beta2 = res.params
    
    return pd.DataFrame([beta1, beta2], ['y', 'x'])

res = qvar(Y, X, 2, 0.3)
res

,const,y_1,y_2,x_1,x_2
y,17.229937,1.390869,-0.369268,-0.589803,0.549461
x,11.392195,0.502069,0.463664,0.345127,-0.324991


## matlab实现
可以参考 matlab 代码：[QVAR 模型](https://github.com/lei940324/econometrics/tree/master/matlab代码/分位数回归/VAR模型)